In [17]:
import torch
import torch.nn.functional as F

import numpy as np

from tqdm import tqdm_notebook as tqdm

from data_generator_helper import generate_synthetic_selection_dataset
from models_new.nalu import NALU
from models_new.nac import NAC

from torchvision import datasets
import torchvision.models as models
import torchvision.utils as vutils
from tensorboardX import SummaryWriter

import datetime
import os

import matplotlib
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D as plt3

from ipywidgets import interactive
from ipywidgets import widgets


from models.mlp import MLP

In [27]:
def reportLoss(loss):
    print(loss)
    
def train_mlp(model, optimizer, x_train, y_train, epochs, batch_size, model_param):
    losses = np.zeros((epochs,len(x_train)//batch_size))
    
    for epoch in range(epochs):
        for batch in range(len(x_train) // batch_size):
            
            model.train()
            optimizer.zero_grad()

            x_batch_train = x_train[batch:(batch+batch_size),:]
            y_batch_train = y_train[batch:(batch+batch_size),:]
            out = model(x_batch_train)

            loss = F.mse_loss(out, y_batch_train)
            
            if loss != loss:
                print("nan detected")
            losses[epoch,batch] = loss
            loss.backward()
            optimizer.step()
        
        if epoch % 1 == 0:
            print(loss)
               
    return test_mlp(model,x_train,y_train),losses

     
    
def test_mlp(model, x_test, y_test):
    model.eval()
    output_test = model(x_test)
    loss = F.mse_loss(output_test, y_test)
    return loss

In [21]:
ini = ['Kai_uni','Xav_norm','Kai_norm','Zeros','Ones']
model_param = "NALU"

net_shape = [[1,100],[2,50],[2,100],[3,50],[3,100]]
net_shape = [[2,50]]
ini = [["NALU"]]
#ini = ini[0]
for shp in net_shape:
    for init in ini:
        test_per_range = 1
        sample_size = 100
        set_size = 100000

        in_dim = sample_size
        hidden_dim = shp[1]
        out_dim = 2
        num_layers = shp[0]

        lr = 0.01
        epochs = 1000
        batch_size = 1
        values = [1] #np.linspace(1,1000,10)
        losses = np.zeros((epochs, set_size//batch_size))
        train_loss = [] #np.zeros((len(values),10))
        test_loss = [] # np.zeros((len(values),10))


        for j, value in tqdm(enumerate(values)):
            model = MLP(num_layers, in_dim, hidden_dim, out_dim, ini)
            optimizer = torch.optim.RMSprop(model.parameters(),lr=lr)

            min_value = 0#-value
            max_value = value

            train_acc = 0
            test_acc = 0
            for k in range(test_per_range):
                i = 0

                x_train, y_train, boundaries = generate_synthetic_selection_dataset(min_value, 
                                                                                    max_value, sample_size, 
                                                                                    set_size, boundaries = None)

                x_test, y_test, _ = generate_synthetic_selection_dataset(min_value, max_value,
                                                                 sample_size, set_size, boundaries = boundaries)
                #bounds[j][i] = boundaries


                train_loss,losses = train_mlp(model, optimizer, x_train, y_train, epochs, batch_size, model_param)
                test_loss  = test_mlp(model, x_test, y_test)

                filename = "nntest_" + model_param +"_"+ str(init) +"_"+ "test_" + str(k) +"_"+ str(num_layers) + "_" + str(hidden_dim) + "_" + str(set_size) 
                print(filename,losses[-1,-1])
                
                t = [[param for param in children.parameters()] for children in model.model.children()]
                #for children in model.model.children():
                #    print("Children:")
                #    print(children)
                #    plist = [param for param in children.parameters()]
                #    print("Plist:")
                #    print(plist)
                np.save(filename,(losses[-1,-1],t,boundaries))



HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

nntest_NALU_['NALU']_test_0_2_50_1000 0.004288416355848312
nntest_NALU_['NALU']_test_1_2_50_1000 0.10299801826477051
nntest_NALU_['NALU']_test_2_2_50_1000 0.06978246569633484
nntest_NALU_['NALU']_test_3_2_50_1000 0.0033828439190983772
nntest_NALU_['NALU']_test_4_2_50_1000 0.04298591986298561
nntest_NALU_['NALU']_test_5_2_50_1000 0.006954716052860022
nntest_NALU_['NALU']_test_6_2_50_1000 0.01935393549501896
nntest_NALU_['NALU']_test_7_2_50_1000 0.01746761053800583
nntest_NALU_['NALU']_test_8_2_50_1000 0.022643879055976868
nntest_NALU_['NALU']_test_9_2_50_1000 0.2977808713912964


In [28]:
init = ['Kai_uni']
shp = [3,50]


test_per_range = 1
sample_size = 100
set_size = 100000

in_dim = sample_size
hidden_dim = shp[1]
out_dim = 2
num_layers = shp[0]

lr = 0.001
epochs = 100
batch_size = 1
losses = np.zeros((epochs, set_size//batch_size))
train_loss = [] #np.zeros((len(values),10))
test_loss = [] # np.zeros((len(values),10))

model = MLP(num_layers, in_dim, hidden_dim, out_dim, ini)
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

min_value = 0#-value
max_value = 1

x_train, y_train, boundaries = generate_synthetic_selection_dataset(min_value, 
                                                                    max_value, sample_size, 
                                                                    set_size, boundaries = None)

x_test, y_test, _ = generate_synthetic_selection_dataset(min_value, max_value,
                                                 sample_size, set_size, boundaries = boundaries)


train_loss,losses = train_mlp(model, optimizer, x_train, y_train, epochs, batch_size, model_param)
test_loss  = test_mlp(model, x_test, y_test)


tensor(0.0050, grad_fn=<MseLossBackward>)
tensor(0.0001, grad_fn=<MseLossBackward>)
tensor(0.0004, grad_fn=<MseLossBackward>)
tensor(0.0002, grad_fn=<MseLossBackward>)
tensor(0.0009, grad_fn=<MseLossBackward>)
tensor(0.0004, grad_fn=<MseLossBackward>)
tensor(0.0001, grad_fn=<MseLossBackward>)
tensor(0.0007, grad_fn=<MseLossBackward>)
tensor(0.0006, grad_fn=<MseLossBackward>)
tensor(0.0000, grad_fn=<MseLossBackward>)
tensor(0.0002, grad_fn=<MseLossBackward>)
tensor(0.0002, grad_fn=<MseLossBackward>)
tensor(0.0004, grad_fn=<MseLossBackward>)
tensor(0.0010, grad_fn=<MseLossBackward>)
tensor(0.0001, grad_fn=<MseLossBackward>)


KeyboardInterrupt: 

In [38]:
valu = [[0,1],[-0.001,0.001],[0,100],[-100,100],[-1000,1000]]
for val in valu:

    min_value = val[0]#-value
    max_value = val[1]

    x_test, y_test, _ = generate_synthetic_selection_dataset(min_value, max_value,
                                                     sample_size, set_size, boundaries = boundaries)

    test_loss  = test_mlp(model, x_test, y_test)
    print(test_loss.data.numpy() / np.max(x_test.data.numpy()))

0.0003001852
586.4125
0.03669976
363.1091
3636.069
